In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
from scipy import stats
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Output_Files/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_SubsetC_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_SubsetC_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time = [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [6]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [7]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [8]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [9]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [10]:
# Function to Calculate Gridbox Size
def Grid_Size(Grid_Count):
    Gridbox = (0.3 * 111.32) ** 2
    Area = Grid_Count * Gridbox
    return (Area)

In [11]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [12]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [13]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [14]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [15]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [16]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [17]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [18]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [19]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [20]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [21]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [22]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    Code_List = DF_Storm["Code"]
    Name_List = DF_Storm["Name"]
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Code": Code_List, "Name": Name_List, \
    "Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP": SLP, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [23]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [24]:
# Find Precip Rate From Compo File
def Precip_Rate(Compo_nc, Compo_Index):
    Precip_ms = numpy.array(Compo_nc.snap_PRECT[int(Compo_Index)])
    Precip_mmhr = Precip_ms * 3600 * 1000
    return (Precip_mmhr)

In [25]:
# Find Precipitable Water From Compo File
def Precip_Water(Compo_nc, Compo_Index):
    Precipitable_Water = numpy.array(Compo_nc.snap_TMQ[int(Compo_Index)])
    return (Precipitable_Water)

In [26]:
# Find Outgoing Longwave Radiation and Cloud Top Temperature From Compo File
def Cloud_Temp(Compo_nc, Compo_Index):
    Outgoing_Longwave = numpy.array(Compo_nc.snap_FLUT[int(Compo_Index)])
    Sigma = 5.67 * 10**-8
    Cloud_Temp_K = (Outgoing_Longwave / (0.95 * Sigma)) ** 0.25
    Cloud_Temp_C = Cloud_Temp_K - 273.15
    return (Cloud_Temp_C)

In [27]:
# Find Surface Temperature From Compo File
def Temp_Surface(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_TS[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [28]:
# Find 850hPa Temperature From Compo File
def Temp_850hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T850[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [29]:
# Find 500hPa Temperature From Compo File
def Temp_500hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T500[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [30]:
# Find 200hPa Temperature From Compo File
def Temp_200hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T200[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [31]:
# Find 500hPa Vertical Velocity From Compo File
def Omega_500hPa(Compo_nc, Compo_Index):
    Vert_Velo = numpy.array(Compo_nc.snap_OMEGA500[int(Compo_Index)])
    return (Vert_Velo)

In [32]:
# Find 200hPa Zonal Winds From Compo File
def U_200hPa(Compo_nc, Compo_Index):
    Zonal_Wind = numpy.array(Compo_nc.snap_U200[int(Compo_Index)])
    return (Zonal_Wind)

In [33]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
    SLP = DF_Storm_Compo["SLP"]
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((6,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
# Count Number of Data Points With Windspeed Above 13,18,25,33m/s
        Snap_Sort = numpy.sort(Snap_850.ravel())
        Count_13 = len(Snap_Sort[Snap_Sort >= 13])
        Count_18 = len(Snap_Sort[Snap_Sort >= 18])
        Count_25 = len(Snap_Sort[Snap_Sort >= 25])
        Count_33 = len(Snap_Sort[Snap_Sort >= 33])
        Wind_Field_Info[2][k] = Grid_Size(Count_13) * 10**-3
        Wind_Field_Info[3][k] = Grid_Size(Count_18) * 10**-3
        Wind_Field_Info[4][k] = Grid_Size(Count_25) * 10**-3
        Wind_Field_Info[5][k] = Grid_Size(Count_33) * 10**-3
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    DF_Storm_Compo["13m/s"] = Wind_Field_Info[2]
    DF_Storm_Compo["18m/s"] = Wind_Field_Info[3]
    DF_Storm_Compo["25m/s"] = Wind_Field_Info[4]
    DF_Storm_Compo["33m/s"] = Wind_Field_Info[5]
    return (DF_Storm_Compo)

In [34]:
# Find Precipitation Variables
def Precip_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
#
# Create Array to Store Data
    Precip_Field_Info = numpy.zeros((7,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Precip Snap From Compo_nc
        Snap_Precip = Precip_Rate(Compo_nc, Compo_Index[k])
# Find Maximum Precip Rate
        Max_Precip = numpy.max(Snap_Precip)
        Precip_Field_Info[0][k] = Max_Precip
# Count Number of Data Points With Precip Rate Above 1, 5, 10mm/hr
        Snap_Sort = numpy.sort(Snap_Precip.ravel())
        Count_05 = len(Snap_Sort[Snap_Sort >= 0.5])
        Count_1 = len(Snap_Sort[Snap_Sort >= 1])
        Count_5 = len(Snap_Sort[Snap_Sort >= 5])
        Count_10 = len(Snap_Sort[Snap_Sort >= 10])
        Precip_Field_Info[1][k] = Grid_Size(Count_05) * 10**-3
        Precip_Field_Info[2][k] = Grid_Size(Count_1) * 10**-3
        Precip_Field_Info[3][k] = Grid_Size(Count_5) * 10**-3
        Precip_Field_Info[4][k] = Grid_Size(Count_10) * 10**-3
#
# Find Precipitable Water Snap From Compo_nc
        Snap_Precip_Water = Precip_Water(Compo_nc, Compo_Index[k])
# Find Maximum Precipitable Water
        Max_Precip_Water = numpy.max(Snap_Precip_Water)
        Precip_Field_Info[5][k] = Max_Precip_Water
# Find Areal Precipitable Water Total
        Precip_Water_Total = numpy.sum(Snap_Precip_Water.ravel())
        Precip_Field_Info[6][k] = Grid_Size(Precip_Water_Total) * 10**-6
# 
# Add Precip Field Info Into DF Storm Compo
    DF_Storm_Compo["Max Precip Rate"] = Precip_Field_Info[0]
    DF_Storm_Compo["0.5mm/hr"] = Precip_Field_Info[1]
    DF_Storm_Compo["1mm/hr"] = Precip_Field_Info[2]
    DF_Storm_Compo["5mm/hr"] = Precip_Field_Info[3]
    DF_Storm_Compo["10mm/hr"] = Precip_Field_Info[4]
    DF_Storm_Compo["Max Precip Water"] = Precip_Field_Info[5]
    DF_Storm_Compo["Total Precip Water"] = Precip_Field_Info[6]
    return (DF_Storm_Compo)

In [35]:
# Find Temperature Variables
def Temp_Vars_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
#
# Create Array to Store Data
    Temp_Field_Info = numpy.zeros((8,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Each Variable From Compo_nc
        Snap_Temp_Cloud = Cloud_Temp(Compo_nc, Compo_Index[k])
        Snap_Temp_Sfc = Temp_Surface(Compo_nc, Compo_Index[k])
        Snap_Temp_850hPa = Temp_850hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_500hPa = Temp_500hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_200hPa = Temp_200hPa(Compo_nc, Compo_Index[k])
        Snap_Omega_500hPa = Omega_500hPa(Compo_nc, Compo_Index[k])
        Snap_U_200hPa = U_200hPa(Compo_nc, Compo_Index[k])
#
# Find Minimum Cloud Top Temperature
        Min_Temp_Cloud = numpy.min(Snap_Temp_Cloud)
# Find Mean Surface, 850hPa, 500hPa, 200hPa Temperatures Within 1 Lat/Lon of Storm Center
        Mean_Temp_Sfc = numpy.mean(Snap_Temp_Sfc[17:23,17:23])
        Mean_Temp_850hPa = numpy.mean(Snap_Temp_850hPa[17:23,17:23])
        Mean_Temp_500hPa = numpy.mean(Snap_Temp_500hPa[17:23,17:23])
        Mean_Temp_200hPa = numpy.mean(Snap_Temp_200hPa[17:23,17:23])
        Mean_U_200hPa = numpy.mean(Snap_U_200hPa[17:23,17:23])
# Find Maximum Rising and Sinking Vertical Velocity
        Min_Omega = numpy.min(Snap_Omega_500hPa) * -1
        Max_Omega = numpy.max(Snap_Omega_500hPa)
#
# Add To Array
        Temp_Field_Info[0][k] = Min_Temp_Cloud
        Temp_Field_Info[1][k] = Mean_Temp_Sfc
        Temp_Field_Info[2][k] = Mean_Temp_850hPa
        Temp_Field_Info[3][k] = Mean_Temp_500hPa
        Temp_Field_Info[4][k] = Mean_Temp_200hPa
        Temp_Field_Info[5][k] = Mean_U_200hPa
        Temp_Field_Info[6][k] = Min_Omega
        Temp_Field_Info[7][k] = Max_Omega
#
# Add To DF Storm Compo
    DF_Storm_Compo["Min Cloud Temp"] = Temp_Field_Info[0]
    DF_Storm_Compo["Sfc Temp"] = Temp_Field_Info[1]
    DF_Storm_Compo["850hPa Temp"] = Temp_Field_Info[2]
    DF_Storm_Compo["500hPa Temp"] = Temp_Field_Info[3]
    DF_Storm_Compo["200hPa Temp"] = Temp_Field_Info[4]
    DF_Storm_Compo["200hPa U"] = Temp_Field_Info[5]
    DF_Storm_Compo["Max Rising"] = Temp_Field_Info[6]
    DF_Storm_Compo["Max Sinking"] = Temp_Field_Info[7]
    return (DF_Storm_Compo)

In [36]:
# Create New Data DF
def DF_Data_Compo(Data_DF, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
    ABC_List = ET_DF["ABC"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Composite_Data(Code_List[n], Data_DF, Compo_DF)
# Find Which Compo nc To Use
        if ABC_List[n] == "A":
            Compo_nc = Compo_nc_A
        elif ABC_List[n] == "B":
            Compo_nc = Compo_nc_B
        elif ABC_List[n] == "C":
            Compo_nc = Compo_nc_C
# Apply Functions For Finding Wind Field and Precip Field
        DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Precip_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Temp_Vars_Find(DF_Storm_Compo, Compo_nc)
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0:
            if DF_Storm_Compo["Storm Phase"][len(DF_Storm_Compo)-1] == "Extratropical":
# Combine DF Storm Compos
                try:
                    Data_Compo = pandas.concat([Data_Compo, DF_Storm_Compo])
                except:
                    Data_Compo = DF_Storm_Compo.copy()
    Data_Compo_Final = Data_Compo.reset_index().drop("index", axis=1)
    return (Data_Compo_Final)

In [37]:
# Create New ET DF
def DF_ET_Compo(Data_Compo, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        ET_Storm = Find_Storm(ET_DF, Code_List[n])
        DF_Storm_Compo = Find_Storm(Data_Compo, Code_List[n])
# Find ET Begin and ET Complete Time
        Trop_Peak_Time = ET_Storm["Trop Peak Time"][0]
        Begin_Time = ET_Storm["ET Begin Time"][0]
        Compl_Time = ET_Storm["ET Complete Time"][0]
        DF_Trop_Peak = DF_Storm_Compo[DF_Storm_Compo["Time"] == Trop_Peak_Time].reset_index()
        DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
        DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0 and len(DF_Trop_Peak) and len(DF_Begin) > 0 and len(DF_Compl) > 0:
# Combine ET Storm Compos
            ET_Storm_Compo = Find_ET_Compo(Code_List[n], ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl)
            try:
                ET_Compo = pandas.concat([ET_Compo, ET_Storm_Compo])
            except:
                ET_Compo = ET_Storm_Compo.copy()
        else:
            print (Code_List[n], len(DF_Trop_Peak), len(DF_Begin), len(DF_Compl))
    ET_Compo_Final = ET_Compo.reset_index().drop("index", axis=1)
    return (ET_Compo_Final)

In [38]:
def Find_ET_Compo(Code, ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl):
    ET_Storm_Compo = ET_Storm[["Code", "Name", "Trop Peak Time", "ET Begin Time", "ET Complete Time", \
    "Trop Peak SLP", "ET Begin SLP", "ET Complete SLP"]].copy()
    Vars = ["850hPa Winds", "13m/s", "18m/s", "25m/s", "33m/s", \
    "0.5mm/hr", "1mm/hr", "5mm/hr", "10mm/hr", "Max Precip Rate", "Max Precip Water", "Total Precip Water", \
    "Min Cloud Temp", "Sfc Temp", "850hPa Temp", "500hPa Temp", "200hPa Temp", "200hPa U", \
    "Max Rising", "Max Sinking"]
    for m in range(len(Vars)):
        Var = Vars[m]
        Trop_Peak_Var = str("Trop Peak " + Var)
        Begin_Var = str("ET Begin " + Var)
        Compl_Var = str("ET Complete " + Var)
        ET_Storm_Compo[Trop_Peak_Var] = DF_Trop_Peak[Var][0]
        ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
        ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
    return (ET_Storm_Compo)

In [39]:
Control_Data_Compo = DF_Data_Compo(Control_Data, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [40]:
Control_ET_Compo = DF_ET_Compo(Control_Data_Compo, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [41]:
RCP45_Data_Compo = DF_Data_Compo(RCP45_Data, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [42]:
RCP45_ET_Compo = DF_ET_Compo(RCP45_Data_Compo, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [43]:
RCP85_Data_Compo = DF_Data_Compo(RCP85_Data, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [44]:
RCP85_ET_Compo = DF_ET_Compo(RCP85_Data_Compo, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [45]:
# Calculate 25%, Median, 75% Percentiles
def Percentile(Array):
    Percent_25 = round(numpy.nanpercentile(Array, 25), 1)
    Median = round(numpy.nanmedian(Array), 1)
    Percent_75 = round(numpy.nanpercentile(Array, 75), 1)
    return ([Percent_25, Median, Percent_75])

In [46]:
# Calculate Statistical Significance Using KS Test
def KS_Test(Control_Array, RCP45_Array, RCP85_Array):
    P_Val_RCP45 = round(stats.kstest(Control_Array, RCP45_Array)[1], 3)
    P_Val_RCP85 = round(stats.kstest(Control_Array, RCP85_Array)[1], 3)
    return (P_Val_RCP45, P_Val_RCP85)

In [47]:
# Create DataFrame to Store Percentiles Data
def Percentile_DF(Var, Control_Array, RCP45_Array, RCP85_Array):
    Control_Percentiles = Percentile(Control_Array)
    RCP45_Percentiles = Percentile(RCP45_Array)
    RCP85_Percentiles = Percentile(RCP85_Array)
    P_Vals = KS_Test(Control_Array, RCP45_Array, RCP85_Array)
    Control_Percentiles.append(1.000)
    RCP45_Percentiles.append(P_Vals[0])
    RCP85_Percentiles.append(P_Vals[1])
    DF = pandas.DataFrame({"Var": [Var, Var, Var, Var], "Percentile": ["25%", "Median", "75%", "P Val"], \
    "Control": Control_Percentiles, "RCP4.5": RCP45_Percentiles, "RCP8.5": RCP85_Percentiles})
    return (DF)

In [48]:
# Create DataFrame For Output
def Create_Output_DF(Control_ET, RCP45_ET, RCP85_ET, Vars):
    for i in range(len(Vars)):
        for j in range(3):
            if j == 0:
                Var = "Trop Peak " + Vars[i]
            elif j == 1:
                Var = "ET Begin " + Vars[i]
            elif j == 2:
                Var = "ET Complete " + Vars[i]
            DF = Percentile_DF(Var, Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
            if i == 0 and j == 0:
                Output_DF = DF.copy()
            else:
                Output_DF = pandas.concat([Output_DF, DF])
    return (Output_DF)

In [49]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, \
["13m/s", "18m/s", "25m/s", "33m/s"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 13m/s,25%,683.7,804.400,732.700
1,Trop Peak 13m/s,Median,1054.0,1217.300,1002.600
2,Trop Peak 13m/s,75%,1419.8,1668.200,1950.600
3,Trop Peak 13m/s,P Val,1.0,0.321,0.286
0,ET Begin 13m/s,25%,1030.5,1339.500,1258.100
1,ET Begin 13m/s,Median,1524.6,1704.200,1805.700
2,ET Begin 13m/s,75%,2068.9,2497.400,2730.200
3,ET Begin 13m/s,P Val,1.0,0.247,0.100
0,ET Complete 13m/s,25%,1819.0,1907.400,2050.500
1,ET Complete 13m/s,Median,2481.5,2452.000,2738.000


In [50]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, \
["0.5mm/hr", "1mm/hr", "5mm/hr", "10mm/hr"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 0.5mm/hr,25%,438.3,320.900,327.300
1,Trop Peak 0.5mm/hr,Median,578.8,519.200,571.000
2,Trop Peak 0.5mm/hr,75%,925.7,747.500,856.000
3,Trop Peak 0.5mm/hr,P Val,1.0,0.078,0.127
0,ET Begin 0.5mm/hr,25%,442.8,470.100,412.700
1,ET Begin 0.5mm/hr,Median,645.8,612.900,533.100
2,ET Begin 0.5mm/hr,75%,830.9,821.700,764.500
3,ET Begin 0.5mm/hr,P Val,1.0,0.979,0.123
0,ET Complete 0.5mm/hr,25%,414.9,433.300,455.000
1,ET Complete 0.5mm/hr,Median,537.6,580.500,552.100


In [51]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, \
["Max Precip Rate", "Max Precip Water", "Total Precip Water"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Max Precip Rate,25%,32.3,28.700,38.200
1,Trop Peak Max Precip Rate,Median,43.3,47.100,50.800
2,Trop Peak Max Precip Rate,75%,49.4,61.100,66.200
3,Trop Peak Max Precip Rate,P Val,1.0,0.074,0.001
0,ET Begin Max Precip Rate,25%,17.3,22.300,19.100
1,ET Begin Max Precip Rate,Median,28.3,30.500,33.900
2,ET Begin Max Precip Rate,75%,40.0,47.600,57.800
3,ET Begin Max Precip Rate,P Val,1.0,0.255,0.147
0,ET Complete Max Precip Rate,25%,9.8,11.100,13.700
1,ET Complete Max Precip Rate,Median,13.9,14.300,20.200


In [52]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, \
["Min Cloud Temp", "Sfc Temp", "850hPa Temp", "500hPa Temp", "200hPa Temp"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Min Cloud Temp,25%,-71.9,-68.600,-70.200
1,Trop Peak Min Cloud Temp,Median,-67.6,-66.300,-65.400
2,Trop Peak Min Cloud Temp,75%,-62.4,-62.700,-59.700
3,Trop Peak Min Cloud Temp,P Val,1.0,0.155,0.292
0,ET Begin Min Cloud Temp,25%,-67.0,-66.500,-63.300
1,ET Begin Min Cloud Temp,Median,-61.2,-59.700,-57.500
2,ET Begin Min Cloud Temp,75%,-54.8,-54.200,-50.800
3,ET Begin Min Cloud Temp,P Val,1.0,0.755,0.123
0,ET Complete Min Cloud Temp,25%,-55.5,-55.400,-54.900
1,ET Complete Min Cloud Temp,Median,-52.2,-50.000,-50.000


In [53]:
Output_DF = Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, \
["850hPa Winds", "18m/s", "33m/s", "5mm/hr", "Max Precip Rate", "Max Precip Water", \
"850hPa Temp", "500hPa Temp", "200hPa Temp", "200hPa U"])
Output_DF

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 850hPa Winds,25%,46.1,48.200,51.500
1,Trop Peak 850hPa Winds,Median,56.6,57.900,59.500
2,Trop Peak 850hPa Winds,75%,65.0,68.100,67.800
3,Trop Peak 850hPa Winds,P Val,1.0,0.762,0.347
0,ET Begin 850hPa Winds,25%,38.5,42.700,37.200
...,...,...,...,...,...
3,ET Begin 200hPa U,P Val,1.0,0.093,0.484
0,ET Complete 200hPa U,25%,14.2,14.500,13.200
1,ET Complete 200hPa U,Median,21.4,21.300,22.100
2,ET Complete 200hPa U,75%,28.6,30.700,28.200


In [54]:
# Output DF to csv File
def Output_File(DF, File_Name):
    DF.to_csv(Output_Diri+File_Name)

In [55]:
Output_File(Output_DF, 'Composites_Table.csv')